# Analyzing Historical Stock/Revenue Data and Building a Dashboard


In [23]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go


In [24]:
# Question 1: Extracting Tesla Stock Data Using yfinance
def get_tesla_stock_data():
    tesla = yf.Ticker("TSLA")
    tesla_data = tesla.history(period="max")
    tesla_data.reset_index(inplace=True)
    return tesla_data

tesla_stock_data = get_tesla_stock_data()

# Display first 5 rows to verify
tesla_stock_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [25]:
# Question 2: Extracting Tesla Revenue Data Using Webscraping
def get_tesla_revenue():
    url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"

    # Adding a user-agent often helps avoid being blocked
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/87.0.4280.66 Safari/537.36"
        )
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Locate the revenue table
    revenue_table = soup.find("table", {"class": "historical_data_table table"})
    if not revenue_table:
        raise ValueError("Tesla revenue table not found on the webpage")

    # Parse rows
    rows = revenue_table.find_all("tr")
    tesla_revenue = []
    for row in rows[1:]:
        cols = row.find_all("td")
        if len(cols) == 2:
            date = cols[0].text.strip()
            revenue = cols[1].text.strip().replace(",", "").replace("$", "")
            tesla_revenue.append([date, revenue])

    tesla_revenue_df = pd.DataFrame(tesla_revenue, columns=["Date", "Revenue"])
    # Remove any empty revenue entries
    tesla_revenue_df = tesla_revenue_df[tesla_revenue_df['Revenue'] != ""]
    return tesla_revenue_df

tesla_revenue_data = get_tesla_revenue()

# Display first 5 rows to verify
tesla_revenue_data.head()

,Date,Revenue
0,2023,96773
1,2022,81462
2,2021,53823
3,2020,31536
4,2019,24578


In [26]:
# Question 3: Extracting GameStop Stock Data Using yfinance
def get_gamestop_stock_data():
    gamestop = yf.Ticker("GME")
    gamestop_data = gamestop.history(period="max")
    gamestop_data.reset_index(inplace=True)
    return gamestop_data

gamestop_stock_data = get_gamestop_stock_data()

# Display first 5 rows to verify
gamestop_stock_data.head()



,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683251,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666417,1.666417,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


In [27]:
# Question 4: Extracting GameStop Revenue Data Using Webscraping
def get_gamestop_revenue():
    url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"

    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/87.0.4280.66 Safari/537.36"
        )
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Locate the revenue table
    revenue_table = soup.find("table", {"class": "historical_data_table table"})
    if not revenue_table:
        raise ValueError("GameStop revenue table not found on the webpage")

    # Parse rows
    rows = revenue_table.find_all("tr")
    gamestop_revenue = []
    for row in rows[1:]:
        cols = row.find_all("td")
        if len(cols) == 2:
            date = cols[0].text.strip()
            revenue = cols[1].text.strip().replace(",", "").replace("$", "")
            gamestop_revenue.append([date, revenue])

    gamestop_revenue_df = pd.DataFrame(gamestop_revenue, columns=["Date", "Revenue"])
    gamestop_revenue_df = gamestop_revenue_df[gamestop_revenue_df['Revenue'] != ""]
    return gamestop_revenue_df

gamestop_revenue_data = get_gamestop_revenue()

# Display first 5 rows to verify
gamestop_revenue_data.head()


,Date,Revenue
0,2024,5273
1,2023,5927
2,2022,6011
3,2021,5090
4,2020,6466


In [28]:
# Question 5 & 6: Creating Dashboards
def create_dashboard(stock_data, revenue_data, title):
    # Convert the 'Date' column to datetime in both dataframes
    stock_data['Date'] = pd.to_datetime(stock_data['Date'])
    revenue_data['Date'] = pd.to_datetime(revenue_data['Date'])

    # Create a new Figure
    fig = go.Figure()

    # Add Stock Price trace
    fig.add_trace(
        go.Scatter(
            x=stock_data['Date'],
            y=stock_data['Close'],
            name='Stock Price',
            yaxis='y1'
        )
    )

    # Add Revenue trace
    fig.add_trace(
        go.Scatter(
            x=revenue_data['Date'],
            y=revenue_data['Revenue'].astype(float),
            name='Revenue',
            yaxis='y2'
        )
    )

    # Create second y-axis
    fig.update_layout(
        title=title,
        xaxis=dict(domain=[0.1, 0.9]),
        yaxis=dict(
            title='Stock Price',
            titlefont=dict(color='#1f77b4'),
            tickfont=dict(color='#1f77b4')
        ),
        yaxis2=dict(
            title='Revenue',
            titlefont=dict(color='#ff7f0e'),
            tickfont=dict(color='#ff7f0e'),
            anchor='x',
            overlaying='y',
            side='right'
        ),
        legend_title='Legend'
    )

    fig.show()


In [29]:
# Create Tesla Dashboard
create_dashboard(tesla_stock_data, tesla_revenue_data, "Tesla Stock Price vs Revenue")


In [30]:
# Create GameStop Dashboard
create_dashboard(gamestop_stock_data, gamestop_revenue_data, "GameStop Stock Price vs Revenue")
